# 方法設計

方法構成了Julia 程式當中的絕大多數的部份，也是 Julia 在多重分派中重要的角色。我們在設計方法的時候需要考慮的是廣義化，一個好的方法被設計出來，希望他不只能夠應用於少數情況，我們會希望他可以適用於更多的情境。我們希望一個方法可以被設計的夠俱備一般性，如此才能適用更多情境。

In [1]:
function plus(x, y)
    return x + y
end

plus (generic function with 1 method)

以上是一個乍看非常簡單，但是異常糟糕的方法實作。在 API 的部份，並沒有去界定 `x` 及 `y` 兩個參數的範圍，如此就允許接收各式各樣型別的物件，這樣很容易導致錯誤。如此一來，使用者也無從得知什麼時候該使用這個方法，便會以為任何型別的物件都可以進行運算，但是事實並非如此。

## API 設計 - 依目的設計

首先，我們需要界定參數型別的範圍。一般來說，需要依據設計者的設計目的而制定。例如：

In [2]:
function plus(x::Vector, y::Vector)
    return x + y
end

plus (generic function with 2 methods)

以上的實作多了一些資訊，但仍舊不足。`plus` 牽涉到的是 `Vector` 當中值的運算，但這邊並沒有去界定值的型別，我們可以利用參數化來處理，最廣義的形式是這個樣子。

In [3]:
function plus(x::Vector{T}, y::Vector{S}) where {T,S}
    return x + y
end

plus (generic function with 2 methods)

我們用參數化來處理，但是並沒有增加任何的限制，所以我們可以在後面加上型別的限制。

In [4]:
function plus(x::Vector{T}, y::Vector{S}) where {T<:Number,S<:Number}
    return x + y
end

plus (generic function with 3 methods)

如此我們完成了對 API 的設計，這是一個可以對一維陣列做加法運算的方法，陣列當中的值要是 `Number` 的子型別。

## 依型別所支援的方法設計

接下來，我們需要檢查方法所接收的參數與內部實作之間的關係。參數有其型別，而我們也在 API 上定義的了型別，型別有各自的方法。我們需要確認我們所設計的 `plus` 中所使用的方法都是有支援的。

使用到的 API：

* `+`

在我們的範例中，`plus` 使用到的方法只有一個，就是 `+`。`+` 會直接作用在 `x` 及 `y` 身上，我們需要確認他們的型別 `Vector{T}` 及 `Vector{S}` 是有支援 `+` 的。這件事我們一般可以由官方文件查詢得到，或是我們可以直接執行並測試我們的實作得到驗證。

In [5]:
plus([1,2,3], [4,5,6])

3-element Array{Int64,1}:
 5
 7
 9

如果使用的是自己設計的方法或是套件，請再次確認被使用到的 API 是有實作的。

## 函式參數的預設值

考慮一個情境，我們在計算變異數的時候容許資料是中心化的（centralization，資料減去其平均值）或是尚未中心化。這樣可以方便使用者指定要中心化的平均值給函式計算。大概會寫出以下這樣的函式。

In [6]:
using Statistics: mean

function variance(a::Array, m=nothing)
    if !isnothing(m)
        m = mean(a, dims=1)
        b = a .- m
    else
        b = a
    end
    c = mean(b.^2, dims=1)
    c
end

variance (generic function with 2 methods)

一般會設置一個變數 `m` 來接收平均值，當沒有平均值時就將 `a` 指定給 `b`。這樣的作法會讓程式碼過於冗長。

我們利用 Julia 的語言特性讓計算平均值這件事自動執行，我們將 `m=mean(a, dims=1)` 直接作為參數的預設值使用，這邊值得注意的是，參數的計算依賴 `a`，所以參數 `a` 一定要在 `m` 之前。

In [7]:
function variance(a::Array, m=mean(a, dims=1))
    b = a .- m
    c = mean(b.^2, dims=1)
    c
end

variance (generic function with 2 methods)

作為預設值時，使用者可以不需要輸入要中心化的平均值，函式會自動計算平均並且中心化。

In [8]:
x = rand(5, 10)
variance(x)

1×10 Array{Float64,2}:
 0.0904065  0.0403577  0.0964654  …  0.149273  0.0358055  0.0563326

使用者也可以手動給定平均值，那麼函式就會自動採用使用者輸入的平均值進行中心化。這麼做的同時保持程式碼的簡潔，也不會有效能損失。

#### 函式參數之間的依賴關係

In [9]:
variance(x, zeros(10)')

1×10 Array{Float64,2}:
 0.739134  0.200017  0.338432  0.159864  …  0.383164  0.35531  0.107148

可以在一些可提供預設值或是額外的計算資訊的情境下使用。不過這麼做會讓參數之間有依賴關係，參數之間的依賴關係是後者會依賴前者。我們這邊以一個簡單的例子來解釋這件事。

In [10]:
foo(a::Array, b=size(a), c=sum(b)) = c

foo (generic function with 3 methods)

In [11]:
x = rand(3, 4, 5);
foo(x)

12

如果調換了 `b` 及 `c` 的順序的話，會導致 `b` 尚未定義。

In [12]:
foo(a::Array, c=sum(b), b=size(a)) = c

foo (generic function with 3 methods)

In [13]:
foo(x)

LoadError: UndefVarError: b not defined

這樣的方式也可以用在解開（unpack）數組。如果指定了輸入的參數要是一個矩陣，那麼就會有兩個維度，這時候我們就可以將它的 `size` 解開成 `b` 及 `c`，已利後續利用。

In [14]:
foo(m::Matrix, (b, c)=size(m)) = b

foo (generic function with 5 methods)

In [15]:
y = rand(5, 6)
foo(y)

5

## 設計函式解決核心問題

寫程式是要用來解決特定問題的，我們會為每個問題設計一套演算法來提供相對應的解。在設計演算法之後，需要將這套演算法寫成相對應的廣泛函式。例如，我們想解決資料排序的問題，而這個排序演算法可以為任何種類的資料進行排序，演算法也不在意所排序的資料多寡。

### 廣義演算法

In [16]:
function bsort!(xs::Vector, n::Int)
    for i = 1:n
        for j = 1:(n-i)
            if xs[j] > xs[j+1]
                xs[j], xs[j+1] = xs[j+1], xs[j]
            end
        end
    end
    xs
end

bsort! (generic function with 1 method)

這邊以最簡單的氣泡排序法作為示範，以上是最簡單的氣泡排序法實作。`bsort!` 會接受一個序列 `xs`，並且它會直接挪動這個序列中的元素。排序法當中會需要序列長度的資訊，這邊特別放一個序列長度 `n` 作為參數。理論上，序列長度可以由序列本身推得，所以可以另外給定一個方法是只有接受序列作為參數的版本。

In [17]:
function bsort!(xs::Vector)
    bsort!(xs, length(xs))
end

bsort! (generic function with 2 methods)

In [18]:
bsort!(rand(5))

5-element Array{Float64,1}:
 0.14536432327735405
 0.2734716963054924
 0.27593681453054764
 0.6957413591067212
 0.8383521166917851

### 利用位置參數的預設值自動產生方法

其實以上這兩種方法可以藉由 Julia 的預設參數來達成，也就是藉由給定位置參數的預設值，Julia 編譯器會自動產生這兩種方法。可以將以上兩個方法合併為以下的寫法：

In [19]:
function bsort!(xs::Vector, n::Int=length(xs))
    for i = 1:n
        for j = 1:(n-i)
            if xs[j] > xs[j+1]
                xs[j], xs[j+1] = xs[j+1], xs[j]
            end
        end
    end
    xs
end

bsort! (generic function with 2 methods)

In [20]:
methods(bsort!)

# 2 methods for generic function "bsort!":
[1] bsort!(xs::Array{T,1} where T) in Main at In[19]:1
[2] bsort!(xs::Array{T,1} where T, n::Int64) in Main at In[19]:1

可以由 `methods` 看到已經有這兩種方法的存在。

### 排序序列中的特定範圍

為了將排序演算法廣義化，可以做成給定序列中的一段範圍，從較低的索引 `lo` 到較高的索引 `hi`。排序演算法可以被修改成以下範例：

In [21]:
function bsort!(xs::Vector, lo::Int, hi::Int)
    for i = lo:hi
        for j = lo:(hi-(i-lo+1))
            if xs[j] > xs[j+1]
                xs[j], xs[j+1] = xs[j+1], xs[j]
            end
        end
    end
    xs
end

bsort! (generic function with 3 methods)

In [22]:
x = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
bsort!(x, 5, 10)

10-element Array{Int64,1}:
 10
  9
  8
  7
  1
  2
  3
  4
  5
  6

修改原本的演算法要注意的是，原本的索引值由 `1` 到 `n` 變成了 `lo` 到 `hi`，所以在內層迴圈中，`j` 的範圍會從 `lo` 到 `hi-(i-lo+1)`，其中 `i-lo+1` 是每次迭代找到較大的數字的個數，每迭代一次就會增加一個較大的數字被推到後面。那這個數字就會是從 `lo` 算起，與 `i` 之間的差，再加上一了。

然而，原本對序列整體的排序仍舊是要支援的，所以就讓最低索引為 1，最高索引為序列長度。

In [23]:
bsort!(xs::Vector, n::Int=length(xs)) = sort!(xs, 1, n)

bsort! (generic function with 3 methods)

### 變更比較大小的方式

如果想要更廣義化排序的對象，可以允許比較大小的方式被改變。比較大小的方式可以因為所比較的型別不同，或是定義不同型別上的比較規則而改變。

In [24]:
>(1, 2)

false

在這邊比較的是，如果前者有比後者還大，則將兩者交換。使用的是 `>` 的符號，而 `>` 廣義上來說是一個函式，所以我們可以用函式呼叫的方式來使用它。

In [25]:
gt = (x, y) -> x > y
gt(1, 2)

false

可以進一步利用匿名函式來定義相同的行為，這個函式可以被作為參數輸入到我們的演算法中，並且取代檢查的位置。

In [26]:
function bsort!(xs::Vector, lo::Int, hi::Int, gt=(x, y) -> x > y)
    for i = lo:hi
        for j = lo:(hi-(i-lo+1))
            if gt(xs[j], xs[j+1])
                xs[j], xs[j+1] = xs[j+1], xs[j]
            end
        end
    end
    xs
end

bsort! (generic function with 4 methods)

這邊自定義了一個新的型別 `Foo` 試試看 `bsort!` 的新功能。在 `Foo` 上並沒有定義任何的大小關係，但是我們想要依據 `Foo` 當中的 `x` 欄位的大小進行排序。假設想要排序的範圍是 2~5，給定的大小關係判定方式為 `(x, y) -> x.x > y.x`，也就是，當輸入兩個元素，想要比較兩個元素的大小的時候，就去比較元素的 `x` 欄位的大小關係，作為兩個元素的大小關係。

In [27]:
struct Foo
    x::Int
end

foos = [Foo(6), Foo(5), Foo(4), Foo(3), Foo(2), Foo(1)]

bsort!(foos, 2, 5, (x, y) -> x.x > y.x)

6-element Array{Foo,1}:
 Foo(6)
 Foo(2)
 Foo(3)
 Foo(4)
 Foo(5)
 Foo(1)

可以看到結果顯示新功能是正確的，並且將 `gt=(x, y) -> x > y` 作為參數的預設值，讓原本對於數字的排序功能也會是對的。

In [28]:
bsort!(x, 3, 10)

10-element Array{Int64,1}:
 10
  9
  1
  2
  3
  4
  5
  6
  7
  8

### 對數值的絕對值進行排序 -- 提供轉換

最後，如果想要比較的元素，假設是一個數組，一個數組當中也有若干個元素在裡頭，要怎麼樣可以依據數組當中的某個元素進行排序呢？甚至想要依據元素的絕對值進行排序，這是做得到的嗎？這代表我們排序的對象在進行比較的時候需要做一個轉換，這個轉換，廣義上來說，就是給定一個函式，讓元素可以對應到一個想要排序的值。來看看以下的範例：

In [29]:
function bsort!(xs::Vector, lo::Int, hi::Int, gt=(x, y) -> x > y, by=identity)
    for i = lo:hi
        for j = lo:(hi-(i-lo+1))
            if gt(by(xs[j]), by(xs[j+1]))
                xs[j], xs[j+1] = xs[j+1], xs[j]
            end
        end
    end
    xs
end

bsort! (generic function with 5 methods)

這時候需要多提供一個參數 `by`，在比較元素之前，要先將 `xs[j]` 通過 `by` 函式進行轉換，轉換過後再用 `gt` 進行比較。我們來看看這會有什麼效果。當數組是欲排序的元素時，可以給定最後一個參數為 `x -> x[1]`，讓演算法依據第一個元素進行排序。

In [30]:
xs = [(3, 'a'), (4, 'g'), (1, 'b'), (6, 'v'), (2, 'f')]
bsort!(xs, 1, 5, (x, y) -> x > y, x -> x[1])

5-element Array{Tuple{Int64,Char},1}:
 (1, 'b')
 (2, 'f')
 (3, 'a')
 (4, 'g')
 (6, 'v')

或是給定 `x -> x[2]`，讓演算法依據第二個元素進行排序。

In [31]:
bsort!(xs, 1, 5, (x, y) -> x > y, x -> x[2])

5-element Array{Tuple{Int64,Char},1}:
 (3, 'a')
 (1, 'b')
 (2, 'f')
 (4, 'g')
 (6, 'v')

也可以依據數字的絕對值排序。

In [32]:
bsort!([-2, 0, 5, -7, -1], 1, 5, (x, y) -> x > y, abs)

5-element Array{Int64,1}:
  0
 -1
 -2
  5
 -7

甚至可以組合先前的功能，對含有 `Foo` 型別的數組進行排序。

In [33]:
foos = [(Foo(6), 'd'), (Foo(5), 's'), (Foo(4), 'b'), (Foo(3), 'w'), (Foo(2), 'l'), (Foo(1), 'k')]
bsort!(foos, 1, 6, (x, y) -> x.x > y.x, x -> x[1])

6-element Array{Tuple{Foo,Char},1}:
 (Foo(1), 'k')
 (Foo(2), 'l')
 (Foo(3), 'w')
 (Foo(4), 'b')
 (Foo(5), 's')
 (Foo(6), 'd')

善用 Julia 的位置參數的預設值特性，可以很方便地產生不同種方法。

In [34]:
methods(bsort!)

# 5 methods for generic function "bsort!":
[1] bsort!(xs::Array{T,1} where T) in Main at In[23]:1
[2] bsort!(xs::Array{T,1} where T, n::Int64) in Main at In[23]:1
[3] bsort!(xs::Array{T,1} where T, lo::Int64, hi::Int64) in Main at In[29]:1
[4] bsort!(xs::Array{T,1} where T, lo::Int64, hi::Int64, gt) in Main at In[29]:1
[5] bsort!(xs::Array{T,1} where T, lo::Int64, hi::Int64, gt, by) in Main at In[29]:1

### 關鍵字參數提供可選的參數

在使用這樣的函式，久了之後就會發現，每當需要更改 `by` 的時候，都一定會需要給定 `gt`，因為位置參數的關係，在前面位置的參數是一定要給定的。有沒有什麼方法可以不需要在更改 `by` 的時候，都給定 `gt` 呢？也就是這兩者功能其實沒有相依性，在使用 `by` 的時候往往跟 `gt` 沒有關係。如果要將兩者參數的相依性打破，則可以將兩者變成關鍵字參數。

In [35]:
function bsort!(xs::Vector, lo::Int, hi::Int; gt=(x, y) -> x > y, by=identity)
    bsort!(xs, lo, hi, gt, by)
end

bsort! (generic function with 5 methods)

In [36]:
bsort!(xs, 1, 5; by=x -> x[1])

5-element Array{Tuple{Int64,Char},1}:
 (1, 'b')
 (2, 'f')
 (3, 'a')
 (4, 'g')
 (6, 'v')

如此一來，就可以打破兩者的相依性。通常關鍵字參數會使用在使用者非必要給定的參數上，並且要讓關鍵字參數具有預設值。相反，位置參數則是在使用這個函式時一定要給定的參數。

## 方法之間的關係

如同前面的敘述，目前已經撰寫了一個廣義演算法來解決排序的核心問題，為了提供更多樣的功能，我們在其上增加了不少參數，也讓不同方法之間互相可以呼叫。如同先前的設計，使用者會希望對整個序列做排序，所以需要補上以下的方法。

In [37]:
function bsort!(xs::Vector, n::Int=length(xs); gt=(x, y) -> x > y, by=identity)
    bsort!(xs, 1, n, gt, by)
end

bsort! (generic function with 5 methods)

### 架構方法

最終真正會使用到的方法為以下三個。作為核心的廣義演算法，位置參數就可以不用提供預設值，預設值可以從呼叫它的其他方法提供。核心的廣義演算法儘量使用位置參數，由於關鍵字參數並不會經過多重分派的機制，所以關鍵字參數並沒有辦法提供參數數量及型別的資訊給編譯器進行判斷。

```julia
function bsort!(xs::Vector, lo::Int, hi::Int, gt, by)
    for i = lo:hi
        for j = lo:(hi-(i-lo+1))
            if gt(by(xs[j]), by(xs[j+1]))
                xs[j], xs[j+1] = xs[j+1], xs[j]
            end
        end
    end
    xs
end

function bsort!(xs::Vector, lo::Int, hi::Int; gt=(x, y) -> x > y, by=identity)
    bsort!(xs, lo, hi, gt, by)
end

function bsort!(xs::Vector, n::Int=length(xs); gt=(x, y) -> x > y, by=identity)
    bsort!(xs, 1, n, gt, by)
end
```

接著，來觀察一下方法之間的相依關係。可以發現所有的方法最終都會去呼叫我們所設計的廣義演算法，畢竟解決排序問題的是這個演算法。如果演算法寫的越廣義，應用層面越廣。然而，程式設計師常常需要因應不同的使用場景設計不同的函式，要解決的核心問題是相同的，所以要將不同應用場景的函式導向共同的廣義演算法。

<img src="pics/methods.svg">

在設計一系列方法及演算法的同時，我們可以參考這樣的模式來進行設計。這樣的設計可以大幅降低所需要撰寫的函式數量，並且以單一的核心方法來解決問題。在除錯上也會比較簡單。使用者使用時，參數也會比較簡單易懂。

### 案例研究：similar

`similar` 是一個非常好用的函式，它可以用來創造與指定陣列相似的陣列。舉例而言，先創造一個隨機陣列 `A`。

In [38]:
A = rand(3, 4)

3×4 Array{Float64,2}:
 0.736767  0.69636   0.325344  0.412994
 0.78388   0.641192  0.578538  0.068511
 0.161561  0.711473  0.990804  0.947359

如果想要創造一個跟 `A` 一樣元素型別跟維度的陣列，可以呼叫 `similar(A)`。它會創造出一個數值未初始化的矩陣。

In [39]:
similar(A)

3×4 Array{Float64,2}:
 3.75e-322  2.17e-322  2.77e-322  4.64e-322
 1.4e-322   2.27e-322  3.75e-322  1.3e-322
 1.8e-322   2.57e-322  4.25e-322  6.91561e-310

若是想要將元素型別改成 `Float32`，只要將 `Float32` 加到參數當中即可。

In [40]:
similar(A, Float32)

3×4 Array{Float32,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

`similar` 也允許我們改變矩陣的維度，只要將維度的資訊作為參數傳入即可。

In [41]:
similar(A, 5, 5, 1)

5×5×1 Array{Float64,3}:
[:, :, 1] =
 6.91562e-310  6.91554e-310  6.91554e-310  6.91554e-310  6.91554e-310
 6.91562e-310  6.91554e-310  6.91554e-310  6.91562e-310  6.91554e-310
 6.91562e-310  6.91562e-310  6.91554e-310  6.91554e-310  6.91554e-310
 6.91554e-310  6.91554e-310  6.91554e-310  6.91554e-310  0.0
 6.91554e-310  6.91554e-310  6.91554e-310  6.91554e-310  0.0

同時改變元素型別及矩陣維度也不是問題。

In [42]:
similar(A, Float32, 5, 5, 1)

5×5×1 Array{Float32,3}:
[:, :, 1] =
 3.25344f-15  4.5668f-41   3.26456f-15  0.0          0.0
 4.5668f-41   3.25344f-15  4.5668f-41   3.26456f-15  0.0
 0.0          4.5668f-41   3.25344f-15  4.5668f-41   3.26456f-15
 0.0          0.0          4.5668f-41   3.25344f-15  4.5668f-41
 3.26456f-15  0.0          0.0          4.5668f-41   3.25344f-15

如此神奇的函式是怎麼寫出來的呢？這邊擷取 Julia 語言官方的程式碼進行說明。這邊可以看到 `similar` 是由一系列的方法組成的，而這些方法都會呼叫其他 `similar` 方法，所以似乎這邊的程式碼也依尋著我們先前提到的模式撰寫的。

```julia
similar(a::AbstractArray{T}) where {T}                             = similar(a, T)
similar(a::AbstractArray, ::Type{T}) where {T}                     = similar(a, T, to_shape(axes(a)))
similar(a::AbstractArray{T}, dims::Tuple) where {T}                = similar(a, T, to_shape(dims))
similar(a::AbstractArray{T}, dims::DimOrInd...) where {T}          = similar(a, T, to_shape(dims))
similar(a::AbstractArray, ::Type{T}, dims::DimOrInd...) where {T}  = similar(a, T, to_shape(dims))

similar(a::AbstractArray, ::Type{T}, dims::Tuple{Union{Integer, OneTo}, Vararg{Union{Integer, OneTo}}}) where {T} = similar(a, T, to_shape(dims))

similar(a::AbstractArray, ::Type{T}, dims::Dims{N}) where {T,N}    = Array{T,N}(undef, dims)
```

程式碼取自 Julia 的 Base 模組，網址為 https://github.com/JuliaLang/julia/blob/master/base/abstractarray.jl。

我們可以看到在最底下的方法，正是先前提到的廣義演算法。它會去呼叫 `Array{T,N}(undef, dims)`，藉此可以創造一個未初始化的陣列，並可以藉由 `T` 指定元素型別，藉由 `N` 指定陣列的維度，而實際的維度則是由 `dims` 給定。

```julia
similar(a::AbstractArray, ::Type{T}, dims::Dims{N}) where {T,N}    = Array{T,N}(undef, dims)
```

當我們呼叫 `similar(A)` 的時候，其實是呼叫 `similar(a::AbstractArray{T}) where {T}`。它會藉由參數化方法去取得元素型別 `T`，並且將它作為參數，再次呼叫其他的 `similar` 方法。

```julia
similar(a::AbstractArray{T}) where {T} = similar(a, T)
```

當我們呼叫 `similar(A, Float32)`，其實是呼叫 `similar(a::AbstractArray, ::Type{T}) where {T}`。它會進一步呼叫 `to_shape(axes(a))` 來取得 `a` 的維度資訊，並且呼叫其他的 `similar` 方法。

```julia
similar(a::AbstractArray, ::Type{T}) where {T} = similar(a, T, to_shape(axes(a)))
```

最後，含有元素型別以及維度資訊的方法呼叫，則會呼叫到廣義演算法 `similar(a::AbstractArray, ::Type{T}, dims::Dims{N}) where {T,N}`，進行創造未初始化矩陣。這邊可以看到從一開始，參數當中所含的資訊是比較少的，藉由不同的方法的呼叫去不斷累積資訊，最後呼叫的是廣義演算法來達成目的。

```julia
similar(a::AbstractArray, ::Type{T}, dims::Dims{N}) where {T,N} = Array{T,N}(undef, dims)
```

當呼叫 `similar(A, 5, 5, 1)` 時 `similar(a::AbstractArray{T}, dims::DimOrInd...) where {T}` 會被呼叫。元素型別資訊會被擷取，而維度資訊會透過 `to_shape` 進行整理，並且呼叫廣義演算法。

```julia
similar(a::AbstractArray{T}, dims::DimOrInd...) where {T} = similar(a, T, to_shape(dims))
```

`to_shape` 會將輸入的參數轉換為維度資訊，例如，輸入的是整數讓它維持是整數輸出，如果輸入的是個數組或是 `Dims`，也讓它維持是原樣輸出，輸出的結果會有整數或數組。

```julia
to_shape(::Tuple{}) = ()
to_shape(dims::Dims) = dims
to_shape(dims::DimsOrInds) = map(to_shape, dims)::DimsOrInds
# each dimension
to_shape(i::Int) = i
to_shape(i::Integer) = Int(i)
to_shape(r::OneTo) = Int(last(r))
to_shape(r::AbstractUnitRange) = r
```

這邊我們看到數組是一種 `Dims`。

In [43]:
(2, 3) isa Dims

true

那如果是 CUDA 套件中的 CuArray 要如何支援 `similar` 函式呢？以下一樣擷取 CUDA 套件中的程式碼進行說明。當 CUDA 套件被載入，以下程式碼會自動被載入。

```julia
Base.similar(a::CuArray{T,N}) where {T,N} = CuArray{T,N}(undef, size(a))
Base.similar(a::CuArray{T}, dims::Base.Dims{N}) where {T,N} = CuArray{T,N}(undef, dims)
Base.similar(a::CuArray, ::Type{T}, dims::Base.Dims{N}) where {T,N} = CuArray{T,N}(undef, dims)
```

程式碼取自 JuliaGPU/CUDA.jl 專案，網址為 https://github.com/JuliaGPU/CUDA.jl/blob/master/src/array.jl

`Base.similar(a::CuArray, ::Type{T}, dims::Base.Dims{N})` 是 `similar` 在 CuArray 上的廣義演算法，它會創造一個未初始化的 CuArray。當給定維度資訊時，`Base.similar(a::CuArray{T}, dims::Base.Dims{N}) where {T,N}` 會被呼叫，也是會從中取得元素型別資訊，並且創造一個未初始化的 CuArray。若是只有給定元素型別資訊，Base 中的 `similar(a::AbstractArray, ::Type{T}) where {T}` 會被呼叫，所以覆用了先前的方法。